### Util

#### Ignore warnings

In [1]:
import warnings
warnings.filterwarnings("ignore")

#### Hot encode and decode

In [2]:
import numpy as np
from collections import OrderedDict
from operator import itemgetter


def myHotEncode(input_data, max_vocab=0, vocab2idx=None):
    "Return the hot-vecotor and the vocab2idx."
    if vocab2idx is None:
        vocabFreq = {}
        for i in input_data:
            for j in i:
                if j not in vocabFreq:
                    vocabFreq[j] = 0
                vocabFreq[j] += 1
        vocabFreq = OrderedDict(sorted(vocabFreq.items(), key=itemgetter(1), reverse=True))
        vocab2idx = {}
        count = 0
        for v in vocabFreq:
            count += 1
            if max_vocab > 0 and count > max_vocab:
                break
            vocab2idx[v] = len(vocab2idx)        
    vocabEmbeddings = np.identity(len(vocab2idx), dtype='float32')
    data_ret = []
    for i in input_data:
        i_ = []
        for j in i:
            if j in vocab2idx:
                i_.append(vocabEmbeddings[vocab2idx[j]])
        if len(i_) == 0:
            i_ = np.zeros((1,len(vocab2idx)))
        data_ret.append(sum(i_))
    return data_ret, vocab2idx


def myHotDecode(input_data, vocab2idx):
    "Return the decode as final representation and decode as indexs"
    data_ = []
    data_idx = []
    for i in input_data:
        i_ = []
        i_idx = []
        if len(i) != len(vocab2idx):
            print('Erro:', 'The vocab2idx not fit the input data!')
            return
        for _i, j in enumerate(i):
            if j > 0:
                v = [k for k in vocab2idx if vocab2idx[k]==_i][0]
                i_.append(v)
                i_idx.append(_i)
        data_.append(i_)
        data_idx.append(i_idx)
    return data_, data_idx

#### Treat a list of str

In [3]:
import re


def treat_str_list(input_list):
    ret = []
    for s in input_list:
        ret += [ss.lower().strip() for ss in re.split('[\W_]+', s)]
    return ret

#### Plot confusion matrix

In [4]:
import matplotlib.pyplot as plt
import itertools


def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Load data

In [5]:
import pandas as pd


#data_path = 'features_annotated.csv'
data_path = 'dataset.csv'
df = pd.read_csv(data_path)

In [6]:
import ast


columns = ['attrs', 'label', 'level', 'tag', 'text', 'brother_attrs', 'brother_tag', 'brother_text', 'url', 'site']

last_url = ''
last_site = ''

css_tss = []
id_tss = []
level_tss = []
tag_tss = []
url_tss = []
text_tss = []
site_tss = []
brother_css_tss = []
brother_id_tss = []
brother_tag_tss = []
brother_text_tss = []
css_ts = []
id_ts = []
level_ts = []
tag_ts = []
url_ts = []
text_ts = []
site_ts = []
brother_css_ts = []
brother_id_ts = []
brother_tag_ts = []
brother_text_ts = []

labels = []
label = 'None'

last_level = -1
line = 0

for attr, label_, level, tag, text, b_attr, b_tag, b_text, url, site in df[columns].values:
    attr = ast.literal_eval(attr)
    b_attr = ast.literal_eval(b_attr)
    if url != last_url:
        css_tss.append(css_ts.copy())
        id_tss.append(id_ts.copy())
        level_tss.append(level_ts.copy())
        tag_tss.append(tag_ts.copy())
        text_tss.append(text_ts.copy())
        url_tss.append(last_url)
        site_tss.append(last_site)
        brother_css_tss.append(brother_css_ts.copy())
        brother_id_tss.append(brother_id_ts.copy())
        brother_tag_tss.append(brother_tag_ts.copy())
        brother_text_tss.append(brother_text_ts.copy())
        css_ts = []
        id_ts = []
        level_ts = []
        tag_ts = []
        text_ts = []
        brother_css_ts = []
        brother_id_ts = []
        brother_tag_ts = []
        brother_text_ts = []
        labels.append(label)
        last_url = url
        last_site = site
        label = 'None'
        line = 0
    if not level > last_level:
        css_tss.append(css_ts.copy())
        id_tss.append(id_ts.copy())
        level_tss.append(level_ts.copy())
        tag_tss.append(tag_ts.copy())
        text_tss.append(text_ts.copy())
        url_tss.append(last_url)
        site_tss.append(last_site)
        brother_css_tss.append(brother_css_ts.copy())
        brother_id_tss.append(brother_id_ts.copy())
        brother_tag_tss.append(brother_tag_ts.copy())
        brother_text_tss.append(brother_text_ts.copy())
        css_ts = css_ts[:level]
        id_ts = id_ts[:level]
        level_ts = level_ts[:level]
        tag_ts = tag_ts[:level]
        text_ts = text_ts[:level]
        brother_css_ts = brother_css_ts[:level]
        brother_id_ts = brother_id_ts[:level]
        brother_tag_ts = brother_tag_ts[:level]
        brother_text_ts = brother_text_ts[:level]
        labels.append(label)
        label = 'None'
    if label_ != 'None':
        label = label_
    # Features
    if 'class' in attr:
        css_ts.append(treat_str_list(attr['class']))
        #css_ts += treat_str_list(attr['class'])
    else:
        css_ts.append([])
    if 'class' in b_attr:
        brother_css_ts.append(treat_str_list(b_attr['class']))
        #css_ts += treat_str_list(attr['class'])
    else:
        brother_css_ts.append([])
    aux_list = []
    if 'id' in attr:
        #aux_list.append(treat_str_list([attr['id']]))
        aux_list += treat_str_list([attr['id']])
    if 'name' in attr:
        #aux_list.append(treat_str_list([attr['name']]))
        aux_list += treat_str_list([attr['name']])
    id_ts.append(aux_list)
    aux_list = []
    if 'id' in b_attr:
        #aux_list.append(treat_str_list([attr['id']]))
        aux_list += treat_str_list([b_attr['id']])
    if 'name' in b_attr:
        #aux_list.append(treat_str_list([attr['name']]))
        aux_list += treat_str_list([b_attr['name']])
    brother_id_ts.append(aux_list)
    tag_ts.append([tag])
    brother_tag_ts.append([b_tag])
    level_ts.append([level, line])
    if text != 'None Text' and type(text) == str:
        # ToDo - append ( [len(text)] + TFIDF_vector  ). Perhaps in another code block
        #text_ts.append([len(text)])
        text_ts.append([len(text)] + [text])
    else:
        text_ts.append([0] + [''])
        
    if b_text != 'None Text' and type(b_text) == str:
        brother_text_ts.append([len(b_text)] + [b_text])
    else:
        brother_text_ts.append([0] + [''])
        
    last_level = level
    line += line
    
css_tss.append(css_ts.copy())
id_tss.append(id_ts.copy())
level_tss.append(level_ts.copy())
tag_tss.append(tag_ts.copy())
text_tss.append(text_ts.copy())
url_tss.append(last_url)
labels.append(label) # new
site_tss.append(last_site)
brother_css_tss.append(brother_css_ts.copy())
brother_id_tss.append(brother_id_ts.copy())
brother_tag_tss.append(brother_tag_ts.copy())
brother_text_tss.append(brother_text_ts.copy())
#batches = batches[1:]
css_tss = css_tss[1:] # new
id_tss = id_tss[1:] # new
level_tss = level_tss[1:] # new
tag_tss = tag_tss[1:] # new
text_tss = text_tss[1:] # new
url_tss = url_tss[1:] # new
labels = labels[1:]
brother_css_tss = brother_css_tss[1:]
brother_id_tss = brother_id_tss[1:]
brother_tag_tss = brother_tag_tss[1:]
brother_text_tss = brother_text_tss[1:]
site_tss = site_tss[1:]

### Mapping the features

In [7]:
css_ = []
id_ = []
tag_ = []
text_ = []
for css in css_tss:
    for i in css:
        css_.append(i)
for _id in id_tss:
    for i in _id:
        id_.append(i)
for tag in tag_tss:
    for i in tag:
        tag_.append(i)
for text in text_tss:
    for i in text:
        if i[1] != '':
            text_.append(i[1])
_, css2Idx = myHotEncode(css_, 100)
_, id2Idx = myHotEncode(id_, 100)
_, tag2Idx = myHotEncode(tag_, 100)

In [8]:
%%time
# CSS classes
for i1, sample in enumerate(css_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=css2Idx)
    css_tss[i1] = sample_
for i1, sample in enumerate(brother_css_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=css2Idx)
    brother_css_tss[i1] = sample_
# ID and Names
for i1, sample in enumerate(id_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=id2Idx)
    id_tss[i1] = sample_
for i1, sample in enumerate(brother_id_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=id2Idx)
    brother_id_tss[i1] = sample_
# Tags
for i1, sample in enumerate(tag_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=tag2Idx)
    tag_tss[i1] = sample_
for i1, sample in enumerate(brother_tag_tss):
    sample_, _ = myHotEncode(sample, vocab2idx=tag2Idx)
    brother_tag_tss[i1] = sample_
css_tss = np.array(css_tss)
id_tss = np.array(id_tss)
level_tss = np.array(level_tss)
tag_tss = np.array(tag_tss)
text_tss = np.array(text_tss)
url_tss = np.array(url_tss)
brother_css_tss = np.array(brother_css_tss)
brother_id_tss = np.array(brother_id_tss)
brother_tag_tss = np.array(brother_tag_tss)
brother_text_tss = np.array(brother_text_tss)
site_tss = np.array(site_tss)

Wall time: 8.88 s


### TF-IDF

In [9]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


vocab_size = 1000
vec = TfidfVectorizer(max_features=vocab_size, stop_words=ENGLISH_STOP_WORDS)
vec.fit(text_)

for i, text in enumerate(text_tss):
    for j, t in enumerate(text):
        text_tss[i][j] = np.concatenate(([t[0]], vec.transform([t[1]]).toarray()[0]))
for i, text in enumerate(brother_text_tss):
    for j, t in enumerate(text):
        brother_text_tss[i][j] = np.concatenate(([t[0]], vec.transform([t[1]]).toarray()[0]))

Wall time: 3min 32s


### Pad the data

In [10]:
from keras.preprocessing.sequence import pad_sequences


css_tss = pad_sequences(css_tss, maxlen=10)
id_tss = pad_sequences(id_tss, maxlen=10)
level_tss = pad_sequences(level_tss, maxlen=10)
tag_tss = pad_sequences(tag_tss, maxlen=10)
text_tss = pad_sequences(text_tss, maxlen=10)

brother_css_tss = pad_sequences(brother_css_tss, maxlen=10)
brother_id_tss = pad_sequences(brother_id_tss, maxlen=10)
brother_tag_tss = pad_sequences(brother_tag_tss, maxlen=10)
brother_text_tss = pad_sequences(brother_text_tss, maxlen=10)

Using TensorFlow backend.


### Mapping the output

In [11]:
labels, label2Idx = myHotEncode([[l] for l in labels])
labels = np.array(labels)

## Model

In [12]:
from keras.models import Model, Sequential
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate,Concatenate


def lstm_model():
    input_css = Input(shape=(None, len(css2Idx)), name='input_css')
    css = LSTM(len(css2Idx), return_sequences=True, name='lstm_css')(input_css)
    
    input_brother_css = Input(shape=(None, len(css2Idx)), name='input_brother_css')
    brother_css = LSTM(len(css2Idx), return_sequences=True, name='lstm_brother_css')(input_brother_css)
    
    input_id = Input(shape=(None, len(id2Idx)), name='input_id')
    _id = LSTM(len(id2Idx), return_sequences=True, name='lstm_id')(input_id)
    
    input_brother_id = Input(shape=(None, len(id2Idx)), name='input_brother_id')
    brother_id = LSTM(len(id2Idx), return_sequences=True, name='lstm_brother_id')(input_brother_id)
    
    input_level = Input(shape=(None, 2), name='input_level')
    level = LSTM(4, return_sequences=True, name='lstm_level')(input_level)
    
    input_tag = Input(shape=(None, len(tag2Idx)), name='input_tag')
    tag = LSTM(len(tag2Idx), return_sequences=True, name='lstm_tag')(input_tag)
    
    input_brother_tag = Input(shape=(None, len(tag2Idx)), name='input_brother_tag')
    brother_tag = LSTM(len(tag2Idx), return_sequences=True, name='lstm_brother_tag')(input_brother_tag)
    
    input_text = Input(shape=(None, (1+vocab_size)), name='input_text')
    text = LSTM((1+vocab_size), return_sequences=True, name='lstm_text')(input_text)
    
    input_brother_text = Input(shape=(None, (1+vocab_size)), name='input_brother_text')
    brother_text = LSTM((1+vocab_size), return_sequences=True, name='lstm_brother_text')(input_brother_text)
    
    output = concatenate([css, brother_css, _id, brother_id, level, tag, brother_tag, text, brother_text], name='concat_inputs')
    output = LSTM(256, name='lstm_concat')(output)
    output = Dense(len(label2Idx), activation='softmax', name='dense_output')(output)
    
    model = Model(inputs=[input_css, input_brother_css, input_id, input_brother_id, input_level, input_tag, input_brother_tag, input_text, input_brother_text], outputs=[output])
    model.compile(loss='categorical_crossentropy', optimizer='nadam')
    #model.summary()
    return model

def lstm_model_old():
    input_css = Input(shape=(None, len(css2Idx)), name='input_css')
    css = LSTM(len(css2Idx), name='lstm_css')(input_css)
    
    input_id = Input(shape=(None, len(id2Idx)), name='input_id')
    _id = LSTM(len(id2Idx), name='lstm_id')(input_id)
    
    input_level = Input(shape=(None, 1), name='input_level')
    level = LSTM(1, name='lstm_level')(input_level)
    
    input_tag = Input(shape=(None, len(tag2Idx)), name='input_tag')
    tag = LSTM(len(tag2Idx), name='lstm_tag')(input_tag)
    
    output = concatenate([css, _id, level, tag], name='concat_inputs')
    output = Dense(256, name='dense_concat')(output)
    output = Dense(len(label2Idx), activation='softmax', name='dense_output')(output)
    model = Model(inputs=[input_css, input_id, input_level, input_tag], outputs=[output])
    model.compile(loss='categorical_crossentropy', optimizer='nadam')
    #model.summary()
    return model

In [ ]:
site_tss

## Test

In [ ]:
%%time
from sklearn.model_selection import GroupKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


accuracy = []
precision = []
recall = []
f1 = []
confusion = []

# Aux print
_, site2Idx = myHotEncode([[u] for u in site_tss])

gp = GroupKFold(n_splits=3)
for train_indexs, test_indexs in gp.split(tag_tss, groups=site_tss):
    
    css_train = css_tss[train_indexs]
    brother_css_train = brother_css_tss[train_indexs]
    id_train = id_tss[train_indexs]
    brother_id_train = brother_id_tss[train_indexs]
    level_train = level_tss[train_indexs]
    tag_train = tag_tss[train_indexs]
    brother_tag_train = brother_tag_tss[train_indexs]
    text_train = text_tss[train_indexs]
    brother_text_train = brother_text_tss[train_indexs]
    
    X_train = [css_train, brother_css_train, id_train, brother_id_train, level_train, tag_train, brother_tag_train, text_train, brother_text_train]
    y_train = labels[train_indexs]
    
    css_test = css_tss[test_indexs]
    brother_css_test = brother_css_tss[test_indexs]
    id_test = id_tss[test_indexs]
    brother_id_test = brother_id_tss[test_indexs]
    level_test = level_tss[test_indexs]
    tag_test = tag_tss[test_indexs]
    brother_tag_test = brother_tag_tss[test_indexs]
    text_test = text_tss[test_indexs]
    brother_text_test = brother_text_tss[test_indexs]
    
    X_test = [css_test, brother_css_test, id_test, brother_id_test, level_test, tag_test, brother_tag_test, text_test, brother_text_test]
    y_test = labels[test_indexs]

    groups_train = site_tss[train_indexs]
    groups_test = site_tss[test_indexs]
    train_g = set()
    test_g = set()
    for g in groups_train:
        train_g.add(site2Idx[g])
    for g in groups_test:
        test_g.add(site2Idx[g])
    print('Train Groups (URLs)', train_g)
    print('Test Groups (URLs)', test_g)
    
    model = KerasClassifier(build_fn=lstm_model)
    model.fit(X_train, y_train, epochs=10, verbose=1)
    #model.fit(tags_train, y_train, epochs=5, verbose=0)

    result = model.predict(X_test)
    #result = model.predict(tags_test)
    y_test = [r.tolist().index(1) for r in y_test]
    acc = accuracy_score(result, y_test)
    accuracy.append(acc)
    p = precision_score(result, y_test, average="macro")
    precision.append(p)
    r = recall_score(result, y_test, average="macro")
    recall.append(r)
    f = f1_score(result, y_test, average="macro")
    f1.append(f)
    confusion.append(confusion_matrix(result, y_test))
    
    print("%s: %.2f %%" % ('Acc', acc*100))
    print("%s: %.2f %%" % ('Precision', p*100))
    print("%s: %.2f %%" % ('Recall', r*100))
    print("%s: %.2f %%" % ('F1', f*100))
    print('')

print("Acc %.2f %% (+/- %.2f %%)" % (np.mean(accuracy)*100, np.std(accuracy)*100))
print("Precision %.2f %% (+/- %.2f %%)" % (np.mean(precision)*100, np.std(precision)*100))
print("Recall %.2f %% (+/- %.2f %%)" % (np.mean(recall)*100, np.std(recall)*100))
print("F1 %.2f %% (+/- %.2f %%)" % (np.mean(f1)*100, np.std(f1)*100))

### Plot confusion matrix

In [ ]:
nc = []
for c in confusion:
    d = len(label2Idx) - len(c)
    c = np.pad(c, (0,d), 'constant')
    nc.append(c)
confusion = nc
cm = np.mean(confusion, axis=0)
plot_confusion_matrix(cm, label2Idx.keys(), normalize=True) 